In [4]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("FEM2.jl")
using .FEM

In [5]:
# Adatok
E = 2e5
ν = 0.3
height = 100
base = 100
depth = 100
elemSize = 10

approxOrder = 1
internalNodes = true
quadElements = false

problem = FEM.Problem("cube")

Main.FEM.Problem("cube", "Solid", 200000.0, 0.3, 7.85e-9, 1.0)

In [16]:
cb = gmsh.model.occ.addBox(0, 0, 0, base, height, depth)

gmsh.model.occ.synchronize()

In [17]:
ee = min(base, height, depth) / 1000
supp = gmsh.model.getEntitiesInBoundingBox(0 - ee, 0 - ee, 0 - ee, 0 + ee, height + ee, depth + ee, 2)
load = gmsh.model.getEntitiesInBoundingBox(base - ee, 0 - ee, 0 - ee, base + ee, height + ee, depth + ee, 2)

1-element Vector{Tuple{Int32, Int32}}:
 (2, 2)

In [18]:
#supp1 = FEM.displacementConstraint([supp[i][2] for i in 1:length(supp)], ux=0, uy=0)
#load1 = FEM.traction([load[i][2] for i in 1:length(load)], fx=0, fy=1, thickness=thickness)

In [19]:
#FEM.generateMesh(problem, sf1, elemSize, approxOrder=approxOrder, algorithm=8, quadrangle=quadElements, internalNodes=internalNodes)

In [20]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [ ]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [ ]:
K = FEM.stiffnessMatrixPlaneStress(problem)

In [ ]:
K, f = FEM.applyBoundaryConditions!(problem, K, [supp1], [load1])

In [ ]:
#det(K)

In [ ]:
q = FEM.solveDisplacement(K, f)

In [ ]:
S = FEM.solveStressPlaneStress(problem, q)

In [ ]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [ ]:
u = FEM.showResultUvec(problem, q, name="uvec", visible=false)
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
sx = FEM.showResultSX(problem, S, name="σx", visible=false, smooth=false)
sy = FEM.showResultSY(problem, S, name="σy", visible=true, smooth=true)
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)

In [ ]:
pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
pp2 = gmsh.model.occ.addPoint(base / 4, height / 2, 0)
pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

pl1 = gmsh.model.occ.addLine(pp1, pp2)
pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


gmsh.model.occ.synchronize()

In [ ]:
FEM.plotOnPath(problem, pl1, sy, 100, name="σred", visible=false);

In [ ]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()